In [1]:
import torch
import torchvision

model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, 1)

device = torch.device('cuda')

model.load_state_dict(torch.load('circuit_model_scalar_01.pth'))

model = model.to(device).eval().half()

In [2]:
import torch2trt
import torch

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True)

In [3]:
torch.save(model_trt.state_dict(), 'circuit_model_scalar_01_trt_half.pth')

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('circuit_model_scalar_01_trt_half.pth'))

In [2]:
import torch
from torch2trt import TRTModule

apex_model_trt = TRTModule()
apex_model_trt.load_state_dict(torch.load('apex_model_trt_half.pth'))

In [3]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=60)

camera.running = True

In [4]:
import cv2
import os
from utils import preprocess

In [5]:
import numpy as np

In [6]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar(
    steering_gain=1.0,
    steering_offset=0.0,
    throttle_gain=-1.0,
    throttle_offset=0.0
)

In [7]:
from IPython.display import display
import ipywidgets

steering_gain = ipywidgets.FloatSlider(min=0.0, max=3.0, value=0.5, step=0.001, description='steering gain')
steering_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='steering offset')
speed_gain = ipywidgets.FloatSlider(min=0.0, max=10.0, value=0.0, step=0.001, description='speed gain')
speed_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='speed offset')

display(
    steering_gain,
    steering_offset,
    speed_gain,
    speed_offset
)

FloatSlider(value=0.5, description='steering gain', max=3.0, step=0.001)

FloatSlider(value=0.0, description='steering offset', max=0.5, min=-0.5, step=0.001)

FloatSlider(value=0.0, description='speed gain', max=10.0, step=0.001)

FloatSlider(value=0.0, description='speed offset', max=0.5, min=-0.5, step=0.001)

In [49]:
# steering 0.55
# speed 6.0
# speed-off 0.04

from utils import preprocess

def execute(change):
    image = change['new']
    data = preprocess(image)
    out = model_trt(data)[0].cpu()
    xy = apex_model_trt(data)[0].cpu()
    
    out = float(out[0])
    car.throttle = np.exp(-15*out) * speed_gain.value + speed_offset.value
    
    x = float(xy[0])
    car.steering = x * steering_gain.value + steering_offset.value
    

In [50]:
execute({'new': camera.value})

In [51]:
camera.observe(execute, names='value')

In [48]:
camera.unobserve_all()

In [13]:
camera.value.mean()

91.83647560586735